1. [20 pts] In this assignment, we will approach the problem with sentiment analysis in our pipeline to classify movie reviews.
Use the Opinion Lexicon from the source: http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
Download the two files, positive-words.txt and negative-words.txt and use them to opinion-score each pooled review (i.e., ignore the original class labels). One approach can be counting the number of occurrences of the lexicon words and scoring appropriately.
(Hint: Quantize the positive and negative frequencies to 0 and 1,
i.e., predict = (opinion_score[:,0] > opinion_score[:,1]) + 0
)
What is the prediction accuracy when done like in the above?

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
import re
import nltk
# Import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [34]:
import csv
def read_file(file_path):
    start_line = 32
    words=[]

    with open(file_path, 'r', encoding="latin-1") as file:
        reader = csv.reader(file, delimiter=',', quotechar='"')
        next(reader)
        for line_number, line in enumerate(reader, start=1):
            if line_number >= start_line:
                words.append(line[0])
    return words
pos_words=read_file('./positive-words.txt')
neg_words=read_file('./negative-words.txt')

In [30]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
def ie_preprocess(document):
    # Sentence Parse
    document = re.sub('<br />', '', document)
    document = re.sub(r'[^\w\s]', '', document)
    sentences = nltk.sent_tokenize(document)
    
    # Word Parse and remove stopwords
    stop_words = set(stopwords.words('english'))
    
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [[word for word in sent if word.lower() not in stop_words and len(word) > 2] for sent in sentences]
    
    return sentences
path = './movie_data.csv'

df = pd.read_csv(path, encoding="utf-8")
df['review'].apply(ie_preprocess)
df.to_csv('preprocessed_movie_data.csv')

0        [[1974, teenager, Martha, Moxley, Maggie, Grac...
1        [[really, like, Kris, Kristofferson, usual, ea...
2        [[SPOILER, read, think, watching, movie, altho...
3        [[people, seen, wonderful, movie, sure, thet, ...
4        [[recently, bought, DVD, forgetting, much, hat...
                               ...                        
49995    [[lets, start, best, building, although, hard,...
49996    [[British, heritage, film, industry, control, ...
49997    [[dont, even, know, begin, one, family, worst,...
49998    [[Richard, Tyler, little, boy, scared, everyth...
49999    [[waited, long, watch, movie, Also, like, Bruc...
Name: review, Length: 50000, dtype: object

In [33]:

# Create a CountVectorizer for text data
tfidf_vectorizer = TfidfVectorizer()

# Prepare your features and labels
X = tfidf_vectorizer.fit_transform(df['review'])
y = df['sentiment']
# Transform the training data using the CountVectorizer

# Initialize and train the Logistic Regression classifier

log_classifier = LinearSVC()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(log_classifier, X, y, cv=cv, scoring='accuracy')

# Report the 10-fold cross-validation performance
mean_accuracy = np.mean(scores)
print("Mean Accuracy:", mean_accuracy)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/l

Mean Accuracy: 0.90302


2. [20 pts] Use only the first 1000 words in Tf-Idf features and compute 10-fold CV classification evaluation performance. We expect a lower score compared to when using the full set of features. Recall that when Tf-Idf features are ranked, the top features exhibit a high information content, since Idf normalizes the document frequencies properly.

3. [20 pts] Add the two columns of positive and negative opinions (e.g., counts) to the 1000 Tf-Idf features in (2.) and re-compute the 10-fold CV classification evaluation. Did opinions from (1.) help?

4. [20 pts] Compare the ranked features as in the previous homework, indicate where the two added opinion features located with respect to the high-ranked Tf-Idf features (as we conducted in the previous assignments)?

5. [20 pts] We were unable to improve the labeling and/or classification as it looks like the IMDB review dataset ground truth is generated from something else.
List some possibilities about the source of the ground truth of original dataset.
Now attempt to extract the voting score from the review (if exists), for example the first review has it.
Outline an approach.